In [ ]:
# Knitr options
### Generic preamble
Sys.setenv(LANG = "en") # For english language
options(scipen = 5) # To deactivate annoying scientific number notation

# rm(list=ls()); graphics.off() # get rid of everything in the workspace
if (!require("knitr")) install.packages("knitr"); library(knitr) # For display of the markdown

### Knitr options
knitr::opts_chunk$set(warning=FALSE,
                     message=FALSE,
                     fig.align="center"
                     )

In [ ]:
### Load packages
library(tidyverse) # Collection of all the good stuff like dplyr, ggplot2 ect.
library(magrittr) # For extra-piping operators (eg. %<>%)



# Introduction

In this session, you will learn the basic grammar of data manipulation, some best-practice advices. Since data manipulation always follows a purpose and requires some understanding of the data at hand, we will also have a first glance data exploration and visualization. However, we will her only cover the very basics and skip most of the details. Again, you will have a dedicated sessions lateron.

In this session, you will learn:

* How to do basic variable filtering, selection, and manipulation
* How to create various types of data summarization
* How to also apply these actions on grouped data
* How to join data from different sources
* How to reshape (pivot) your data

From my experience, this covers ca. 80% of common data manipulation tasks. Sound like fun? Lets get started!

# The 5 core verbs of data-manipulation

So, let's zoom in the `dplyr` functionality and syntax for a moment, before we illustrate it at the following data-manipulation tasks.

At ity's core are 5 central verbs of data/manipulation, which are

1. `filter()` picks cases based on their values.
2. `select()` picks variables based on their names.
3. `arrange()` changes the ordering of the rows.
4. `mutate()` adds new variables that are functions of existing variables
5. `summarise()` reduces multiple values down to a single summary.

These all combine naturally with `group_by()` which allows you to perform any operation "by group". You can learn more about them in `vignette("dplyr")`. As well as these single-table verbs, dplyr also provides a variety of two-table verbs (joins and mutations), which you can learn about in `vignette("two-table")`.


## The dataset
We will explore now a bit with the starwars dataset, which comes together with `dplyr`. First, let's get an intuition what's in it. `head()` shows us the first 10 rows.



In [ ]:
starwars %>% head() # The "startwars" dataset is an integrated dplyr goddie :)


`glimpse()` gives us an overview over all variable types.



In [ ]:
starwars %>% glimpse()


## 1: `filter()`
The verb `filter()` lets you subset a dataframe by rows (observations), meaning the output will filter for only rows which fulfill a certain condition.



In [ ]:
starwars %>%
  filter(species == "Droid")


Conditions can be combined with boolean operators (`&, |`)



In [ ]:
starwars %>%
  filter(homeworld == "Tatooine" & species == "Droid")


## 2: `select()`
The verb `select()` lets you subset a dataframe by column (variable), meaning the output will only contain certain columns in the stated order



In [ ]:
starwars %>%
  select(name, homeworld) %>%
  head(10)


It can also be used to deselect columns with a leading `-`



In [ ]:
starwars %>%
  select(-birth_year, -skin_color) %>%
  head(10)


Lastly, many of `dplyr`'s other conditions and helper functions can be applied to select or deselect columns. More on that later.


In [ ]:
starwars %>%
  select(name, ends_with("color")) %>%
  head(10)


## 3: `arrange()`
The verb `arrange()` defines the way the rows of your dataframe are ordered



In [ ]:
starwars %>%
  arrange(homeworld, desc(mass)) %>% # Note: desc() witll order it descending, so the heaviest one first.
  head(10)


## 4: `mutate()`
The verb `mutate()` lets you manipulate existing variables or create new ones.



In [ ]:
starwars %>%
  mutate(bmi = mass / ((height / 100)  ^ 2), # If you didnt know how to calculate the BMI, now you do...
         mass.rel = mass / max(mass, na.rm = TRUE)) %>%
  select(name:mass, mass.rel, bmi) %>%
  arrange(desc(bmi)) %>%
  head(10)


## 5: `summarize()`
The verb `summarize()` reduces your dataset to one observation, which is summarized according to a defined function.



In [ ]:
starwars %>%
  summarize(min.height = min(height, na.rm = TRUE),
            mean.height = mean(height, na.rm = TRUE),
            max.height = max(height, na.rm = TRUE),
            sd.height = sd(height, na.rm = TRUE) )




## (+1) Pivoting
By now, we worked with the traditional data structure, where every column corresponds to one variable. We also call that a **wide*"** data format. This is in most cases the most efficient and prefered way to keep your data. However, there are some cases, where another dataformat would be more convenient.

For example, for range of graphical as well as numerical summaries, it would be nice to have every observation and every variable to be mapped in a single row. This format is also calles **long** (or in R slang, **tidy**) data. Adittionally, some real life datasets (including a lot of WorldBank, Eurostats, OECD, ect.) data is usually available in a long format. Therefore, weare in need of easily switching between long and wide formats, to **reshape** your data (in Excel slang, people call that **pivoting**).



In [ ]:
fish_encounters %>% head(10)


Lets take a look at a `pivot_wider` example


In [ ]:
# See vignette("pivot") for examples and explanation
fish_encounters %>%
  pivot_wider(names_from = station, values_from = seen) %>%
  head(10)

And now `pivot_longer`


In [ ]:
# See vignette("pivot") for examples and explanation
# Simplest case where column names are character data
relig_income %>%
  head(10)

In [ ]:
relig_income %>%
  pivot_longer(!religion, names_to = "income", values_to = "count") %>%
  head(10)






## (+2) Doing manipulation by group: `group_by`
Finally, `by_group()` is offers the perfect complement to the 5 basic verbs, since it allows to perform aqll the above mentioned tasks sepperate according to used-defined groupings in one or multiple categories. Lets look at some examples:

First of all, it works amazingly well with the `summarize()` verb, producing within group summaries



In [ ]:
starwars %>%
  group_by(homeworld) %>%
  summarize(mean.height = mean(height, na.rm = TRUE)) %>%
  ungroup() %>%
  arrange(desc(mean.height))


Note that it is adviced to finish always such operations with `ungroup()`. Otherwise, the still active grouping might at a later point lead to all kind of unforseen results.

## Your turn

Some little exercises in between: Please solve **Exercise 1** in the corresponding section on `github`




# Two-table verbs
It's rare that a data analysis involves only a single table of data. In practice, you'll normally have many tables that contribute to an analysis, and you need flexible tools to combine them. In `dplyr`, there are three families of verbs that work with two tables at a time:

1. Mutating joins, which add new variables to one table from matching rows in another.
2. Filtering joins, which filter observations from one table based on whether or not they match an observation in the other table.
3. Set operations, which combine the observations in the data sets as if they were set elements.

Among those, mutating joins are the by far most used ones, so we will briefly discuss them here, and cover other types later in the applied part. For questions right away, consult `vignette(two-"two-table")`. Mutating joins allow you to combine variables from multiple tables (merging data). We here assume our data is "tidy" (meaning: rows = observations, columns = variables).

Lets briefly create some example data:


In [ ]:
df1 <- data_frame(x = c(1, 2), y = 2:1)
df2 <- data_frame(x = c(1, 3), a = 10, b = "a")
df1
df2


There are 4 types of mutating join to combine variables from dataframe `x` and `y, which differ in their behaviour when a match is not found. We'll illustrate each with a simple example:

1. `left_join(x, y)` includes all observations in `x`, regardless of whether they match or not. This is the most commonly used join because it ensures that you don't lose observations from your primary table.


In [ ]:
df1 %>% left_join(df2)



2. `right_join(x, y)` includes all observations in `y`. It's equivalent to `left_join(y, x)`, but the columns will be ordered differently.


In [ ]:
df1 %>% right_join(df2)

In [ ]:
df2 %>% left_join(df1)


3. `inner_join(x, y)` only includes observations that match in both `x` and `y`.


In [ ]:
df1 %>% inner_join(df2)


4. `full_join()` includes all observations from `x` and `y`.


In [ ]:
df1 %>% full_join(df2)


The left, right and full joins are collectively know as **outer joins**. When a row doesn't match in an outer join, the new variables are filled in with missing values.

While mutating joins are primarily used to add new variables, they can also generate new observations. If a match is not unique, a join will add all possible combinations (the Cartesian product) of the matching observations:



In [ ]:
df1 <- data_frame(x = c(1, 1, 2), y = 1:3)
df2 <- data_frame(x = c(1, 1, 2), z = c("a", "b", "a"))

df1 %>% left_join(df2)


Note: In case of multiple matches in `y`, adittional rows will be added. Therefore, if that is not the behavior you want, use the `distinct()` function afterwards.

By default, joins are performed on the bases of **all** variables appearing in `x` and `y`. You can also specify one or multiple variables that should be matched by using the `by` argument. This can also be used to match on variables with different names in `x` and `y`.



In [ ]:
df1 <- data_frame(x = c(3, 2, 1), y = 1:3, z = c(1,3,4))
df2 <- data_frame(x = c(1, 2, 3), y = 2:4, q = c("a", "b", "a"))

df1 %>% left_join(df2, by = "x")
df1 %>% left_join(df2, by = c("x", "y"))
df1 %>% left_join(df2, by = c("x" = "y"))



# Your turn

Some little exercises in between: Please solve **Exercise 2** in the corresponding section on `github`


# Endnotes

### References

### Suggestions for further study

* R for Data Science (Grolemund & Wickham)
  * [Chapter 5](https://r4ds.had.co.nz/transform.html)
  * [Chapter 12](https://r4ds.had.co.nz/tidy-data.html)
  * [Chapter 13](https://r4ds.had.co.nz/relational-data.html)
* DataCamp
  * [Working with Data in the Tidyverse](https://learn.datacamp.com/courses/working-with-data-in-the-tidyverse)
  * [Data Manipulation with dplyr](https://learn.datacamp.com/courses/data-manipulation-with-dplyr)
  * [Joining Data with dplyr](https://learn.datacamp.com/courses/joining-data-with-dplyr)


### Session info


In [ ]:
sessionInfo()